In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-04 15:41:24--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  18.1MB/s    in 90s     

2021-01-04 15:42:54 (20.5 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-04 15:42:54--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [2]:
!cp ../drive/MyDrive/slt/late-fusion-duo/*.py signjoey/

In [3]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [4]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

## Hidden features

In [5]:
for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:17<00:00, 91.09it/s]


## Change config files

In [6]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 1024]

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Run

In [7]:
!cp -r '../drive/MyDrive/slt/models/baseline_40' baseline
!cp -r '../drive/MyDrive/slt/models/hidden_features_40' features

In [8]:
import os
import sys
from signjoey.prediction import test

test(cfg_file="configs/sign.yaml", ckpts=["baseline/best.ckpt", "features/best.ckpt"], output_path="")

2021-01-04 15:59:15,632 - ------------------------------------------------------------
2021-01-04 15:59:15,636 - [DEV] partition [RECOGNITION] experiment [BW]: 1
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
2021-01-04 15:59:46,943 - finished in 31.3068s 
2021-01-04 15:59:46,944 - ************************************************************
2021-01-04 15:59:46,945 - [DEV] partition [RECOGNITION] results:
	New Best CTC Decode Beam Size: 1
	WER 52.12	(DEL: 48.12,	INS: 0.51,	SUB: 3.50)
2021-01-04 15:59:46,946 - ************************************************************
2021-01-04 15:59:46,946 - ------------------------------------------------------------
2021-01-04 15:59:46,947 - [DEV] partition [RECOGNITION] exper

translation


2021-01-04 16:18:34,926 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 1 and Alpha: -1
	BLEU-4 19.43	(BLEU-1: 45.81,	BLEU-2: 32.05,	BLEU-3: 24.26,	BLEU-4: 19.43)
	CHRF 43.42	ROUGE 45.36
2021-01-04 16:18:34,927 - ------------------------------------------------------------
2021-01-04 16:22:05,998 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: -1
	BLEU-4 20.67	(BLEU-1: 45.48,	BLEU-2: 32.68,	BLEU-3: 25.36,	BLEU-4: 20.67)
	CHRF 43.33	ROUGE 46.66
2021-01-04 16:22:05,999 - ------------------------------------------------------------
2021-01-04 16:23:06,532 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: 1
	BLEU-4 20.84	(BLEU-1: 45.77,	BLEU-2: 32.92,	BLEU-3: 25.58,	BLEU-4: 20.84)
	CHRF 43.48	ROUGE 46.68
2021-01-04 16:23:06,533 - ------------------------------------------------------------
2021-01-04 16:58:23,468 - ************************************************************
2021-01-04 16: